In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images/N68.xml,319,479,262,295
1,./images/N49.xml,196,352,135,173
2,./images/N225.xml,775,1142,965,1101
3,./images/N239.xml,106,263,238,276
4,./images/N191.xml,317,568,319,397


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./images/N68.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images/N68.jpeg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./images/N68.jpeg',
 './images/N49.jpeg',
 './images/N225.jpeg',
 './images/N239.jpeg',
 './images/N191.jpeg',
 './images/N152.jpeg',
 './images/N82.jpeg',
 './images/N133.jpeg',
 './images/N233.jpeg',
 './images/N160.jpeg',
 './images/N40.jpeg',
 './images/N183.jpeg',
 './images/N103.jpeg',
 './images/N212.jpeg',
 './images/N172.jpeg',
 './images/N178.jpeg',
 './images/N34.jpeg',
 './images/N155.jpeg',
 './images/N88.jpeg',
 './images/N126.jpeg',
 './images/N106.jpeg',
 './images/N184.jpeg',
 './images/N216.jpeg',
 './images/N142.jpeg',
 './images/N164.jpeg',
 './images/N91.jpeg',
 './images/N113.jpeg',
 './images/N215.jpeg',
 './images/N244.jpeg',
 './images/N190.jpeg',
 './images/N130.jpeg',
 './images/N193.jpeg',
 './images/N117.jpeg',
 './images/N174.jpeg',
 './images/N79.jpeg',
 './images/N175.jpeg',
 './images/N97.jpeg',
 './images/N47.jpeg',
 './images/N15.jpeg',
 './images/N70.jpeg',
 './images/N218.jpeg',
 './images/N54.jpeg',
 './images/N75.jpeg',
 './images/N62.jpeg',
 '.

#### verify image and output

In [8]:
file_path = image_path[0]
file_path 

'./images/N68.jpeg'

In [9]:
# img = cv2.imread(file_path)

# cv2.namedWindow('example',cv2.WINDOW_NORMAL)
# cv2.imshow('example',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
# 1093	1396	645	727
# cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
# cv2.namedWindow('example',cv2.WINDOW_NORMAL)
# cv2.imshow('example',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

### Data Preprocessing

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2024-04-13 20:25:01.701221: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 20:25:01.704353: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 20:25:01.745880: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 20:25:02.501790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
labels = df.iloc[:,1:].values

In [13]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [14]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [15]:
X.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [16]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

### Deep Learning Model

In [17]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [18]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [19]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 19,326,754 (73.73 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

block8_5_mixed (Concatenate)    (None, 5, 5, 448)    0           activation_179[0][0]             
                                                                 activation_182[0][0]             
__________________________________________________________________________________________________
block8_5_conv (Conv2D)          (None, 5, 5, 2080)   933920      block8_5_mixed[0][0]             
__________________________________________________________________________________________________
block8_5 (Lambda)               (None, 5, 5, 2080)   0           block8_4_ac[0][0]                
                                                                 block8_5_conv[0][0]              
__________________________________________________________________________________________________
block8_5_ac (Activation)        (None, 5, 5, 2080)   0           block8_5[0][0]                   
__________________________________________________________________________________________________
conv2d_184

Total params: 73,663,490
Trainable params: 19,326,754
Non-trainable params: 54,336,736
__________________________________________________________________________________________________


### model training

In [20]:
from tensorflow.keras.callbacks import TensorBoard

In [21]:
tfb = TensorBoard('object_detection')

In [22]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=400,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - loss: 0.1171 - val_loss: 0.1530
Epoch 2/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 794ms/step - loss: 0.1362 - val_loss: 0.1235
Epoch 3/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 810ms/step - loss: 0.1108 - val_loss: 0.1209
Epoch 4/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 838ms/step - loss: 0.1070 - val_loss: 0.1221
Epoch 5/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 835ms/step - loss: 0.1098 - val_loss: 0.1224
Epoch 6/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 852ms/step - loss: 0.1083 - val_loss: 0.1236
Epoch 7/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 849ms/step - loss: 0.1139 - val_loss: 0.1192
Epoch 8/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 895ms/step - loss: 0.1108 - val_loss: 0.1188
Epoch 9/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 871ms/step - loss: 0.1084 - val_loss: 0.1237
Epoch 10/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 898ms/step - loss: 0.1111 - val_loss: 0.1180
Epoch 11/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 891ms/step - loss: 0.1045 - val_loss: 0.1207
Epoch 12/400
18/18 ━━━━━━━━━━━━━━

In [23]:
# history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
#                     validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

In [25]:
model.save('./models/object_detection.h5')

In [26]:
image_path

['./images/N68.jpeg',
 './images/N49.jpeg',
 './images/N225.jpeg',
 './images/N239.jpeg',
 './images/N191.jpeg',
 './images/N152.jpeg',
 './images/N82.jpeg',
 './images/N133.jpeg',
 './images/N233.jpeg',
 './images/N160.jpeg',
 './images/N40.jpeg',
 './images/N183.jpeg',
 './images/N103.jpeg',
 './images/N212.jpeg',
 './images/N172.jpeg',
 './images/N178.jpeg',
 './images/N34.jpeg',
 './images/N155.jpeg',
 './images/N88.jpeg',
 './images/N126.jpeg',
 './images/N106.jpeg',
 './images/N184.jpeg',
 './images/N216.jpeg',
 './images/N142.jpeg',
 './images/N164.jpeg',
 './images/N91.jpeg',
 './images/N113.jpeg',
 './images/N215.jpeg',
 './images/N244.jpeg',
 './images/N190.jpeg',
 './images/N130.jpeg',
 './images/N193.jpeg',
 './images/N117.jpeg',
 './images/N174.jpeg',
 './images/N79.jpeg',
 './images/N175.jpeg',
 './images/N97.jpeg',
 './images/N47.jpeg',
 './images/N15.jpeg',
 './images/N70.jpeg',
 './images/N218.jpeg',
 './images/N54.jpeg',
 './images/N75.jpeg',
 './images/N62.jpeg',
 '.